In [0]:
!nvidia-smi

!pip install pytorch-nlp
!pip install pytorch-transformers


Wed Jul 31 07:29:54 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Get Data from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
%cd Dropbox-Uploader
!chmod +x dropbox_uploader.sh
!bash dropbox_uploader.sh
!echo "CHTZbyeOjyYAAAAAAAAAfZTfC_d3kTmHXSqg7-941o9Nolj-FEAJbjyUKutEtCSu" > token.txt
!bash dropbox_uploader.sh

!bash dropbox_uploader.sh download imdb_masked_lm.zip
!unzip imdb_masked_lm.zip
!rm imdb_masked_lm.zip



Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 309.19 KiB | 221.00 KiB/s, done.
Resolving deltas: 100% (505/505), done.
/content/Dropbox-Uploader

 This is the first time you run this script, please follow the instructions:

 1) Open the following URL in your Browser, and log in using your account: https://www.dropbox.com/developers/apps
 2) Click on "Create App", then select "Dropbox API app"
 3) Now go on with the configuration, choosing the app permissions and access restrictions to your DropBox folder
 4) Enter the "App Name" that you prefer (e.g. MyUploader245411274017848)

 Now, click on the "Create App" button.

 When your new App is successfully created, please click on the Generate button
 under the 'Generated access token' section, then execute the following command:

 echo "INPUT_YOUR_ACCESS_TOKEN_HERE" > token.txt


 This is the first time you run 

In [0]:
import numpy as np
import torch.nn as nn
import torch
from pytorch_transformers import BertForMaskedLM, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from random import Random
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [0]:
config = BertConfig.from_pretrained('bert-base-uncased')
config.output_hidden_states=True
model = BertForMaskedLM(config)
#model.load_state_dict(torch.load("checkpoint_6000"))
#model = BertForMaskedLM.from_pretrained('bert-base-uncased')
#model = torch.load('pretrained_bert_imdb_final')
model.cuda()

100%|██████████| 313/313 [00:00<00:00, 51259.89B/s]


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterm

In [0]:
#Load preprocessed Data

masked_input_ids = torch.load('kindle_reviews_128/kindle_ids_128')
attention_masks = torch.load('kindle_reviews_128/kindle_input_masks_128')
masked_lm_labels = torch.load('kindle_reviews_128/kindle_masked_labels_128')


In [0]:
train_batch_size = 50

#Load data into dataloader
train_data = TensorDataset(masked_input_ids, attention_masks, masked_lm_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

In [0]:
max_grad_norm = 1.0
# TRAIN loop
model.train()

epochs = 1


for j in range(epochs):
  
  print("Begin Training")
  
  for step, batch in enumerate(train_dataloader):
    
        
    if(step%100==0):
      print("batch: ", step+1)

    batch = tuple(t.to(device) for t in batch)
    b_train_ids, b_input_masks, b_train_labels = batch

    # forward pass
    outputs = model(input_ids=b_train_ids,attention_mask=b_input_masks,masked_lm_labels=b_train_labels)
    loss = outputs[0]

    # backward pass
    loss.backward()

    # gradient clipping
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)

    # update parameters
    optimizer.step()
    model.zero_grad()

    #Save model every 2000 batches:

    if((step)%2000==0):
      torch.save(model,'pretrained_bert_kindle_128_' + str(step))
       

     

Begin Training
batch:  1
batch:  101
batch:  201
batch:  301
batch:  401
batch:  501
batch:  601
batch:  701
batch:  801
batch:  901
batch:  1001
batch:  1101
batch:  1201
batch:  1301
batch:  1401
batch:  1501
batch:  1601
batch:  1701
batch:  1801
batch:  1901
batch:  2001
batch:  2101
batch:  2201
batch:  2301
batch:  2401
batch:  2501
batch:  2601
batch:  2701
batch:  2801
batch:  2901
batch:  3001
batch:  3101
batch:  3201
batch:  3301
batch:  3401
batch:  3501
batch:  3601
batch:  3701
batch:  3801
batch:  3901
batch:  4001
batch:  4101
batch:  4201
batch:  4301
batch:  4401
batch:  4501
batch:  4601
batch:  4701
batch:  4801
batch:  4901
batch:  5001
batch:  5101
batch:  5201
batch:  5301
batch:  5401
batch:  5501
batch:  5601
batch:  5701
batch:  5801
batch:  5901
batch:  6001
batch:  6101
batch:  6201
batch:  6301
batch:  6401
batch:  6501
batch:  6601
batch:  6701
batch:  6801
batch:  6901
batch:  7001
batch:  7101
batch:  7201
batch:  7301
batch:  7401
batch:  7501
batch:  7

In [0]:
torch.save(model,'pretrained_bert_kindle_128_final')